In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 10:05:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
curr_dir = '/home/jdtganding/Documents/data-engineering-zoomcamp/week_5_batch_processing'
df_green = spark.read.parquet(f'{curr_dir}/data/pq/green/*/*')
df_yellow = spark.read.parquet(f'{curr_dir}/data/pq/yellow/*/*')

In [9]:
df_green.createOrReplaceTempView('green')
df_yellow.createOrReplaceTempView('yellow')

In [10]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_green_revenue \
    .repartition(20) \
    .write.parquet(f'{curr_dir}/data/report/revenue/green', mode='overwrite')

df_yellow_revenue \
    .repartition(20) \
    .write.parquet(f'{curr_dir}/data/report/revenue/yellow', mode='overwrite')

We then join the green and yellow revenue tables:

In [24]:
#we will join the dataframe by the hour and zone columns
#we rename the rest accordingly for each service type

df_green_revenue = df_green_revenue.withColumnRenamed('amount', 'green_amount') \
                                   .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue = df_yellow_revenue.withColumnRenamed('amount', 'yellow_amount') \
                                     .withColumnRenamed('number_records', 'yellow_number_records')

In [26]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour','zone'], how='outer')
df_join.show(10)

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999997|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|106.99999999999999|                    6|
|2020-01-01 00:00:00|  13|              null|                null|1214.7999999999997|                   56|
|2020-01-01 00:00:00|  18|               7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  29|  

In [28]:
df_join.write.parquet(f'{curr_dir}/data/report/revenue/total', mode='overwrite')

In [12]:
# from pyspark.sql.types import IntegerType

# df_zones = spark.read.option('header','true').csv('/home/jdtganding/Documents/data-engineering-zoomcamp/week_5_batch_processing/data/taxi+_zone_lookup.csv')
# df_zones = df_zones.withColumn('LocationID', df_zones.LocationID.cast(IntegerType()))
# df_zones.coalesce(1).write.parquet(f'{curr_dir}/data/zones', mode='overwrite')

In [29]:
df_zones = spark.read.parquet(f'{curr_dir}/data/zones/*')
df_zones.show(10)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 10 rows



We finally join the zones dataframe with the revenue total dataframe

In [30]:
df_result = df_join.join(df_zones, df_zones.LocationID==df_join.zone)
df_result.show(10)

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|         3|    Bronx|Allerton/Pelham G...|   Boro Zone|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|   7| 769.7299999999997|                  45| 455.1700000000001|                   38|         7|   Queens|             Astoria|   Boro Zone|
|2020-01-01 00:00:00| 

In [34]:
df_result.drop('zone').write.parquet(f'{curr_dir}/data/report/revenue/revenue-zones')